In [178]:
# Copyright 2020 Anish Singhani
#
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from textwrap import wrap
import hashlib
import sys
from functools import reduce
import random

DEBUG = False


k = [0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
     0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
     0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
     0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
     0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
     0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
     0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
     0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2]

initial_h = [0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a, 0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19]

def generate_message_schedule_array(chunk):
    w = [0] * 64 # Message schedule array

    # Chunk forms the start of the message schedule array
    for i in range(0, 16):
        w[i] = (chunk & int("F" * 8 + "0" * 8 * (15 - i), 16)) >> (32 * (15 - i))

    # Extend to form the rest of the message schedule array
    for i in range(16, 64):
        s0 = right_rotate(w[i - 15], 7) ^ right_rotate(w[i - 15], 18) ^ (w[i - 15] >> 3)
        s1 = right_rotate(w[i - 2], 17) ^ right_rotate(w[i - 2], 19) ^ (w[i - 2] >> 10)
        w[i] = add32(w[i - 16], s0, w[i - 7], s1)

    return w

def compression(a, b, c, d, e, f, g, h, w):
    for i in range(0, 64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ (~e & g)
        temp1 = add32(h, S1, ch, k[i], w[i])
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = add32(S0, maj)

        #print(i, [hex(_) for _ in (a, b, c, d, e, f, g, h)])
        #print()

        h = g
        g = f
        f = e
        e = add32(d, temp1)
        d = c
        c = b
        b = a
        a = add32(temp1, temp2)
                
    return a, b, c, d, e, f, g, h

def add_elementwise(arr1, arr2, add_func = lambda a, b: a+b):
    assert len(arr1) == len(arr2)
    arr_sum = [0] * len(arr1)

    for i in range(len(arr_sum)):
        arr_sum[i] = add_func(arr1[i], arr2[i])

    return arr_sum

def append_binary(values, bits):
    return reduce(lambda acc, x: (acc << bits) | x, values)

def right_rotate(value, bits):
    return ((value >> bits) | (value << (32 - bits))) & 0xFFFFFFFF

def add32(*args):
    return sum(args) % (2**32)

def swap_endianness(number, length):
    s = hex(number)[2:]
    while len(s) < (length * 2):
        s = "0" + s

    s = wrap(s, 2) # Split into bytes
    s = s[::-1] # Reverse byte order
    s = "".join(s) # Merge back
    return int(s, 16)


def pack_list(x, w=32):
    out = 0
    for word in x:
        out = (out << w) | word
    return out

def add_padding(msg, size):
    msg = bin(msg)[2:]
    
    if len(msg) != size*8:
        msg = "0"*(size*8 - len(msg)) + msg
    
    length = len(msg)
    msg = msg + "1"
    pad_len = 512 - ((length + 65) % 512)
    msg = msg + "0"*pad_len
    msg = msg + "{:064b}".format(length)
    final_length = len(msg)
    return int(msg, 2), final_length

def splitwords(x, length, wordsize=32):
    assert wordsize % 8 == 0
    assert length % wordsize == 0
    s = hex(x)[2:]
    while len(s) < (length / 4):
        s = "0" + s

    s = wrap(s, int(wordsize / 4))

    return [int(a, 16) for a in s]


In [186]:
# Single chunk
data = "hello"
data_packed = pack_list([ord(x) for x in data], w=8)
data_padded, final_length = add_padding(data_packed, size=len(data))
assert final_length == 512

data_padded_words = splitwords(data_padded, 512, wordsize=32)
assert pack_list(data_padded_words, w=32) == data_padded

w = generate_message_schedule_array(data_padded)

hashed = compression(*initial_h, w)
hashed = add_elementwise(hashed, initial_h, add_func=add32)

print(data)
print()
print(", ".join(["0x{:08X}l".format(x) for x in data_padded_words]))
print()
print(", ".join(["0x{:08X}l".format(x) for x in w]))
print()
print(", ".join(["0x{:08X}l".format(x) for x in hashed]))
print()
print("{:064x}".format(pack_list(hashed)))

hello

0x68656C6Cl, 0x6F800000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000028l

0x68656C6Cl, 0x6F800000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000000l, 0x00000028l, 0x7594884Cl, 0x6F910000l, 0xD532D15Al, 0xA01BDE7Al, 0x32B37C8Bl, 0x94DA02F9l, 0xD1D853F8l, 0xB69B56C2l, 0x934804C3l, 0x96C77CA5l, 0xA2F94846l, 0x849744EFl, 0x21DD3E6Al, 0x1CA36B00l, 0x3F957A61l, 0x2079CF86l, 0x6BFC7C3Bl, 0x0C209D4Bl, 0x2228225Cl, 0xEE1AD25Bl, 0xC5D09850l, 0x07331882l, 0x977423C5l, 0xA667D6F9l, 0xBC044B46l, 0x95859031l, 0x5C98CA62l, 0x64860C1Bl, 0x7EE7C777l, 0x16C55872l, 0x7CF67D29l, 0x4857C138l, 0xA37F4A81l, 0x35F8FD03l, 0xF16FFCE2l, 0x6D549F9Cl, 0xA12FF314l, 0xF1818DB4l, 0xF9CAD136l, 0xC960D70Cl, 0x3A18F5F8l, 0xF5FDF2BAl, 0xE79A1B2Al, 0x5DEA4345l, 0x701

In [184]:
# Multi chunk
data = "3a57cd30563a57cd30560fb2f7283a7433ece91946ac60f3a57cd30560fb2f7283a7433ece91946ac6b23a53a57cd3a57cd30560fb2f7283a7433ece91946ac630560fb2f7283a7433ece91946ac67cd30560fb2f7283a7433ece91946ac6f7283a3a57cd3056030563a57cd30560fb2f7283a7433ece91946ac60f3a57cd30560fb2f7283a7433ece91946ac6b23a53a57cd3a57cd30560fb2f7283a7433ece91946ac630560fb2f7283a7433ece91946ac67cd30560fb2f7283a7433ece91946ac6f7283a3a57cd30560fb2f7fb2f7283a7433ece91946ac67433ece91946ac6"

data_packed = pack_list([ord(x) for x in data], w=8)
data_padded, final_length = add_padding(data_packed, size=len(data))
assert final_length % 512 == 0

data_padded_words = splitwords(data_padded, 512, wordsize=32)
assert pack_list(data_padded_words, w=32) == data_padded

hash_value = initial_h[:]
for i in range(final_length // 512):
    data_chunk = pack_list(data_padded_words[(16*i):(16*i+16)], w=32)
    w = generate_message_schedule_array(data_chunk)
    


    hashed = compression(*hash_value, w)
    hash_value = add_elementwise(hashed, hash_value, add_func=add32)
    
    """print(", ".join(["0x{:08X}l".format(x) for x in w]))
    print()
    print(", ".join(["0x{:08X}l".format(x) for x in hashed]))
    print()
    print(", ".join(["0x{:08X}l".format(x) for x in hash_value]))
    print("\n\n\n\n---")"""


print(data)
print()
print(", ".join(["0x{:08X}l".format(x) for x in data_padded_words]))
print()
print(", ".join(["0x{:08X}l".format(x) for x in hash_value]))
print()
print("{:064x}".format(pack_list(hash_value)))

3a57cd30563a57cd30560fb2f7283a7433ece91946ac60f3a57cd30560fb2f7283a7433ece91946ac6b23a53a57cd3a57cd30560fb2f7283a7433ece91946ac630560fb2f7283a7433ece91946ac67cd30560fb2f7283a7433ece91946ac6f7283a3a57cd3056030563a57cd30560fb2f7283a7433ece91946ac60f3a57cd30560fb2f7283a7433ece91946ac6b23a53a57cd3a57cd30560fb2f7283a7433ece91946ac630560fb2f7283a7433ece91946ac67cd30560fb2f7283a7433ece91946ac6f7283a3a57cd30560fb2f7fb2f7283a7433ece91946ac67433ece91946ac6

0x33613537l, 0x63643330l, 0x35363361l, 0x35376364l, 0x33303536l, 0x30666232l, 0x66373238l, 0x33613734l, 0x33336563l, 0x65393139l, 0x34366163l, 0x36306633l, 0x61353763l, 0x64333035l, 0x36306662l, 0x32663732l, 0x38336137l, 0x34333365l, 0x63653931l, 0x39343661l, 0x63366232l, 0x33613533l, 0x61353763l, 0x64336135l, 0x37636433l, 0x30353630l, 0x66623266l, 0x37323833l, 0x61373433l, 0x33656365l, 0x39313934l, 0x36616336l, 0x33303536l, 0x30666232l, 0x66373238l, 0x33613734l, 0x33336563l, 0x65393139l, 0x34366163l, 0x36376364l, 0x33303536l, 0x30666232l, 0x

In [166]:
len("128941289273492374982341111145645643534534sakldjsalkdjaskldjaslk")

64